In [ ]:
import keras
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2


In [ ]:
(x_train, y_train), (x_val, y_val) = cifar100.load_data()

# One-hot encoding
y_train = to_categorical(y_train, 100)
y_val = to_categorical(y_val, 100)


169001437/169001437 [==============================] - 2s 0us/step


In [ ]:
vgg19_model = VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

model = Sequential()
model.add(vgg19_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))

# freeze
vgg19_model.trainable = False


In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

import cv2
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda x: cv2.resize(x, (32, 32), interpolation=cv2.INTER_CUBIC)
)

train_iterator = datagen.flow(x_train, y_train, batch_size=128)
val_iterator = datagen.flow(x_val, y_val, batch_size=128)

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.001,
    verbose=1,
    restore_best_weights=True
)

model.fit(
    train_iterator,
    epochs=30,
    validation_data=val_iterator,
    callbacks=[early_stop]
)

Epoch 1/30
391/391 [==============================] - 10s 24ms/step - loss: 3.6104 - categorical_accuracy: 0.1596 - val_loss: 3.0912 - val_categorical_accuracy: 0.2549
Epoch 2/30
391/391 [==============================] - 9s 23ms/step - loss: 3.1032 - categorical_accuracy: 0.2395 - val_loss: 2.8969 - val_categorical_accuracy: 0.2849
Epoch 3/30
391/391 [==============================] - 9s 23ms/step - loss: 2.9461 - categorical_accuracy: 0.2703 - val_loss: 2.8137 - val_categorical_accuracy: 0.3064
Epoch 4/30
391/391 [==============================] - 9s 23ms/step - loss: 2.8536 - categorical_accuracy: 0.2898 - val_loss: 2.7600 - val_categorical_accuracy: 0.3081
Epoch 5/30
391/391 [==============================] - 9s 24ms/step - loss: 2.7818 - categorical_accuracy: 0.3042 - val_loss: 2.7108 - val_categorical_accuracy: 0.3199
Epoch 6/30
391/391 [==============================] - 9s 24ms/step - loss: 2.7264 - categorical_accuracy: 0.3135 - val_loss: 2.6783 - val_categorical_accuracy: 0.32